In [2]:
import argparse
import uproot
import glob
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [1]:
tree = "Delphes"
branches = ["Track.PID", "Track.Charge", "Track.PT", "Track.P", "Track.Eta", "Track.Phi", "Track.EtaOuter", "Track.PhiOuter", "Track.XOuter", "Track.YOuter"]
pionPID = 211 # plus/minus

In [3]:
def getDataFromRootFile(file):
    with uproot.open(file) as f:
        # load the branches
        temp = {}
        for branch in branches:
            temp[branch] = np.array(ak.flatten(f[tree][branch].array()))
        
        # selection
        cut = (abs(temp["Track.PID"])==pionPID)

        # apply selection
        for branch in branches:
            temp[branch] = temp[branch][cut]
        
        # Original Method:
        #   phi = alpha - pi
        #   theta = beta - pi
        beta1 = temp["Track.PhiOuter"]+np.pi

        # Randomly generate beta between minimum and maximum allowed values
        betamin = 1.31272
        betamax = 1.82887
        beta2 = np.random.uniform(betamin, betamax,size=len(temp["Track.PID"]))
        
        # Determine yentry and ylocal based on beta
        yentry = 30/np.tan(beta2)
        ylocal=yentry+0.08125

        # Get angle of hit location with respect to barrel
        gamma0 = np.arctan2(temp["Track.YOuter"],temp["Track.XOuter"])

        # Get angle of sensor center with respect to barrel
        gamma=gamma0+yentry/30

        # Define unit vector of track at tracker edge with respect to barrel
        theta=2*np.arctan(np.exp(-temp["Track.EtaOuter"]))
        phi=temp["Track.PhiOuter"]
        x=np.sin(theta)*np.cos(phi)
        y=np.sin(theta)*np.sin(phi)
        z=np.cos(theta)

        # Transform into rotated coordinate system (sensor coordinate system sort of)
        xp=x*np.cos(gamma)+y*np.sin(gamma)
        yp=-x*np.sin(gamma)+y*np.cos(gamma)

        beta3=np.arctan2(yp,xp)+np.pi/2
        alpha=np.arctan2(np.sqrt(xp**2+yp**2),z)
        
    df = pd.DataFrame()
    df["Eta"] = temp["Track.Eta"]
    df["EtaOuter"] = temp["Track.EtaOuter"]
    df["Phi"] = temp["Track.Phi"]
    df["PhiOuter"] = temp["Track.PhiOuter"]
    df["Beta1"] = beta1
    df["Beta2"] = beta2
    df["Beta3"] = beta3
    df["Gamma"] = gamma
    df["alpha"]=alpha
    
    return df

In [3]:
files = "/home/elizahoward/cmspix28-mc-sim/temp/"
file = files+"minbias_0.00_0.10_GeV.root"

In [5]:
df = getDataFromRootFile(file)


In [24]:
np.max(df.Beta3)

3.1404004122122204

In [25]:
np.min(df.Beta3)

-3.1414332489236867

In [4]:
with uproot.open(file) as f:
        # load the branches
        temp = {}
        for branch in branches:
            temp[branch] = np.array(ak.flatten(f[tree][branch].array()))
        
        # selection
        cut = (abs(temp["Track.PID"])==pionPID)

        # apply selection
        for branch in branches:
            temp[branch] = temp[branch][cut]
        

In [5]:
temp["Track.Charge"]

array([-1, -1,  1, ...,  1,  1,  1], dtype=int32)